In [1]:
# pd.read

In [2]:
import altair as alt
from vega_datasets import data
# Starter: https://altair-viz.github.io/gallery/one_dot_per_zipcode.html
# Vega datasets to fix dependency issue: https://github.com/altair-viz/vega_datasets
# Since the data is more than 5,000 rows we'll import it from a URL
# Zip codes State Range: https://www.structnet.com/instructions/zip_min_max_by_state.html 60001-62999
# https://altair-viz.github.io/user_guide/transform/filter.html
source = data.zipcodes.url
alt.Chart(source).transform_calculate(
    "leading digit", alt.expr.substring(alt.datum.zip_code, 0, 1)
).transform_filter(
   alt.FieldRangePredicate(field="zip_code", range=[60001, 62999])
).mark_circle(size=3).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size='zip_code:Q',
    tooltip='zip_code:N'
).project(
    type='albersUsa'
).properties(
    width=650,
    height=400
)
# alt.FieldRangePredicate(field="year", range=[1950, 1960])

alt.Chart(...)

In [3]:
!wget https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv

--2025-11-24 04:56:03--  https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1994275 (1.9M) [text/plain]
Saving to: ‘building_inventory.csv.6’

building_inventory. 100%[===================>]   1.90M  --.-KB/s    in 0.01s   

2025-11-24 04:56:04 (136 MB/s) - ‘building_inventory.csv.6’ saved [1994275/1994275]



In [4]:
# Ideas: Graph of density of buildings by zip code
# Filter by building status

In [5]:
# Load data
import pandas as pd
building_df = pd.read_csv("./building_inventory.csv")
# https://docs.vultr.com/python/third-party/pandas/Series/between

illinois_buildings_df = building_df[building_df["Zip code"].between(60001,62999)]
illinois_buildings_df=illinois_buildings_df.rename(columns={"Zip code": "zip_code"})

In [6]:
# pd.read_csv(source)
# Add logic to lookup latitude longitude for zip code
zipcodes_df = pd.read_csv(source)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
# zipcodes_df = zipcodes_df.set_index("zip_code")
zipcodes_df = zipcodes_df.set_index('zip_code')
# # https://stackoverflow.com/questions/71462389/efficient-way-to-lookup-values-in-other-dataframe
illinois_buildings_df['latitude'] = illinois_buildings_df['zip_code'].map(zipcodes_df['latitude'])
illinois_buildings_df['longitude'] = illinois_buildings_df['zip_code'].map(zipcodes_df['longitude'])

In [7]:
summary_series = illinois_buildings_df.groupby(["zip_code","Bldg Status"]).size()
s_df_2 = summary_series.reset_index(name="count")
summary_df =summary_series.reset_index(name='count').pivot(
    index = 'zip_code',
    columns = 'Bldg Status',
    values = 'count'
).fillna(0)
summary_df.head()

s_df = summary_df.reset_index()
s_df_2

,zip_code,Bldg Status,count
0,60005,In Use,4
1,60007,In Use,6
2,60016,In Use,4
3,60018,In Use,1
4,60025,In Use,1
...,...,...,...
563,62988,In Use,23
564,62992,In Use,2
565,62995,Abandon,1
566,62995,In Use,91


In [8]:
il_geojson = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/refs/heads/master/il_illinois_zip_codes_geo.min.json"

In [9]:
import geopandas as gpd

il_geo = gpd.read_file(il_geojson)
il_geo

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,17,62359,1762359,B5,G6350,S,10360074,7921,+40.0338795,-091.2014548,N,"POLYGON ((-91.1829 40.02688, -91.18258 40.0267..."
1,17,62376,1762376,B5,G6350,S,195448214,10146176,+40.1205484,-091.4348909,N,"POLYGON ((-91.37741 40.18398, -91.37743 40.183..."
2,17,62312,1762312,B5,G6350,S,238419841,152277,+39.7073534,-091.0242372,N,"POLYGON ((-90.99116 39.79476, -90.99114 39.794..."
3,17,62365,1762365,B5,G6350,S,72664377,50492,+39.7964029,-091.1651242,N,"POLYGON ((-91.24216 39.78642, -91.24213 39.790..."
4,17,62246,1762246,B5,G6350,S,369861147,3361007,+38.8903641,-089.4274969,N,"POLYGON ((-89.44209 38.78472, -89.44343 38.784..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1379,17,60484,1760484,B5,G6350,S,17428533,5723,+41.4436154,-087.7083544,N,"POLYGON ((-87.74789 41.44159, -87.74635 41.441..."
1380,17,62948,1762948,B5,G6350,S,48619075,1054532,+37.8171381,-089.0306782,N,"MULTIPOLYGON (((-89.02172 37.75127, -89.02181 ..."
1381,17,62921,1762921,B5,G6350,S,1971949,26511,+37.8019117,-089.0842620,N,"POLYGON ((-89.08973 37.80368, -89.08973 37.803..."
1382,17,62915,1762915,B5,G6350,S,313171,0,+37.7836078,-089.1196056,N,"MULTIPOLYGON (((-89.11599 37.78789, -89.1153 3..."


In [10]:
# alt.Chart(il_geo).mark_geoshape(
#     stroke='black',
#     strokeWidth=1
# ).transform_lookup(
#         lookup='ZCTA5CE10',
#         from_= alt.LookupData(data=s_df, key='zip_code',
#                          fields=["Abandon","In Progress","In Use"]),
#     default=0
#     ).project(
#     type='albersUsa'
# ).encode(
#     color = "Abandon:Q",
#     # color = 'black',
#     tooltip= 'ZCTA5CE10:Q'
# )

^Graph takes too long to render (can merge table)
missing zipcodes idk and I am too tired. (My guess is look up fails somehow even though we are doing default?? No clue) Visualization is better with dots

In [11]:
import pandas as pd
zip_codes_df = pd.read_csv(source)
il_zip_codes_df = zip_codes_df[zip_codes_df["zip_code"].between(60001, 62999)]
# Merge right cause we are drawing points and only want those which actually have buildings
final_df = il_zip_codes_df.merge(s_df, how = 'inner',on='zip_code')

In [12]:
# final_df.set_index("zip_code")[final_df.set_index("zip_code")["latitude"].isnull()]

In [13]:
alt.data_transformers.disable_max_rows()
# https://altair-viz.github.io/user_guide/transform/aggregate.html
# https://medium.com/dataexplorations/creating-choropleth-maps-in-altair-eeb7085779a1
# ^ Idea: Use two layers 

select = alt.selection_point(name="select", on="click",fields=["usage_status"])
chart1 = alt.Chart(final_df).mark_circle(size=5).transform_fold(
    ["Abandon", "In Use", "In Progress"],
    as_ = ["usage_status", "usage_count"]
).transform_filter(select).transform_aggregate(
    usage_total_count = "sum(usage_count)",
    groupby = ['zip_code','longitude','latitude']
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip= [alt.Tooltip('usage_total_count:Q',title='Number of Buildings:'),alt.Tooltip('zip_code:Q',title='Zip Code:'),alt.Tooltip('latitude:Q',title='Latitude:'),alt.Tooltip('longitude:Q',title='Longitude:')],
    size= alt.Size('usage_total_count:Q',title="Number of Buildings")
).project(
    type='albersUsa'
).properties(
    width=800,
    height=400
)
chart2 = alt.Chart(final_df).mark_bar().transform_fold(
    ["Abandon", "In Use", "In Progress"],
    as_ = ["usage_status", "usage_count"]
).transform_aggregate(
    usage_total_count = "sum(usage_count)",
    groupby = ['usage_status']
).encode(
    x=alt.X("usage_status:N",title="Usage Status"),
    y=alt.Y('usage_total_count:Q',scale=alt.Scale(type='symlog'),title="Count"),
    fillOpacity=alt.when(select).then(alt.value(1)).otherwise(alt.value(0.3))
).add_params(select).properties(
    width = 300,
    height=400
)
# https://altair-viz.github.io/gallery/interactive_bar_select_highlight.html
chart2|chart1

alt.HConcatChart(...)

In [14]:
(chart2|chart1).save("submission.json")

In [15]:
# https://www.structnet.com/instructions/zip_min_max_by_state.html